In [16]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [17]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


QUESTIONS:

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [26]:
df = pd.read_sql_query('''SELECT name as facility_name, total_revenue
                    FROM (SELECT name, SUM(CASE WHEN Bookings.memid = 0
                        THEN Facilities.guestcost * slots
                        ELSE Facilities.membercost * slots END) as total_revenue
                        FROM Bookings JOIN Members ON Bookings.memid = Members.memid
                        JOIN Facilities ON Bookings.facid = Facilities.facid
                        GROUP BY name) as facility_revenue
                    WHERE total_revenue < 1000
                    ORDER BY total_revenue DESC''', engine)
print(df.head())

   facility_name  total_revenue
0     Pool Table            270
1  Snooker Table            240
2   Table Tennis            180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [46]:
df1 = pd.read_sql_query('''SELECT m1.memid AS ID, m1.surname || ', ' || m1.firstname as member_name, 
                            m2.surname || ', ' || m2.firstname as recommender_name FROM Members as m1
                            INNER JOIN Members as m2 ON m1.recommendedby = m2.memid 
                            WHERE m1.firstname NOT LIKE 'GUEST' AND m2.firstname NOT LIKE 'GUEST'  
                            ORDER BY member_name''', engine)
print(df1)

    ID               member_name    recommender_name
0   15           Bader, Florence    Stibbons, Ponder
1   12               Baker, Anne    Stibbons, Ponder
2   16            Baker, Timothy     Farrell, Jemima
3    8               Boothe, Tim         Rownam, Tim
4    5           Butters, Gerald       Smith, Darren
5   22              Coplin, Joan      Baker, Timothy
6   36            Crumpet, Erica        Smith, Tracy
7    7               Dare, Nancy    Joplette, Janice
8   20          Genting, Matthew     Butters, Gerald
9   35                Hunt, John  Purview, Millicent
10  11              Jones, David    Joplette, Janice
11  26            Jones, Douglas        Jones, David
12   4          Joplette, Janice       Smith, Darren
13  21           Mackenzie, Anna       Smith, Darren
14  10             Owen, Charles       Smith, Darren
15  17             Pinker, David     Farrell, Jemima
16  30        Purview, Millicent        Smith, Tracy
17  27         Rumney, Henrietta    Genting, M

Q12: Find the facilities with their usage by member, but not guests 

In [47]:
 df2 = pd.read_sql_query('''SELECT f.name AS facility_name, b.memid as member_ID, COUNT(f.name) AS facility_usage_count
        FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid
        INNER JOIN Members AS m ON b.memid = m.memid
        GROUP BY f.name, m.memid
        HAVING m.memid <> 0''', engine)
print(df2)

       facility_name  member_ID  facility_usage_count
0    Badminton Court          1                   132
1    Badminton Court          2                    32
2    Badminton Court          3                     4
3    Badminton Court          5                    20
4    Badminton Court          6                     2
..               ...        ...                   ...
197   Tennis Court 2         24                    11
198   Tennis Court 2         27                     1
199   Tennis Court 2         28                     1
200   Tennis Court 2         30                     1
201   Tennis Court 2         35                     4

[202 rows x 3 columns]


Q13: Find the facilities usage by month, but not guests

In [51]:
df3 = pd.read_sql_query('''SELECT f.name AS facility_name, strftime('%m', b.starttime) AS month, COUNT(f.name) AS facility_usage_count
        FROM Bookings AS b
        INNER JOIN Facilities AS f
        USING (facid)
        INNER JOIN Members AS m
        USING (memid)
        WHERE memid <>0
        GROUP BY  f.name, strftime('%m', b.starttime)''', engine)
print(df3)

      facility_name month  facility_usage_count
0   Badminton Court    07                    51
1   Badminton Court    08                   132
2   Badminton Court    09                   161
3    Massage Room 1    07                    77
4    Massage Room 1    08                   153
5    Massage Room 1    09                   191
6    Massage Room 2    07                     4
7    Massage Room 2    08                     9
8    Massage Room 2    09                    14
9        Pool Table    07                   103
10       Pool Table    08                   272
11       Pool Table    09                   408
12    Snooker Table    07                    68
13    Snooker Table    08                   154
14    Snooker Table    09                   199
15     Squash Court    07                    23
16     Squash Court    08                    85
17     Squash Court    09                    87
18     Table Tennis    07                    48
19     Table Tennis    08               